# Importação de bibliotecas importantes


In [17]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary


## Definição da nossa rede de Multilayer Perceptron (MLP)
Nossa rede herda de nn.Module e é construída com especificações para o tamanho das camadas de entrada, camada oculta e camada de saída.
Para o dataset MNIST, o tamanho da camada de saída é 10, pois estamos classificando dígitos de 0 a 9.
A classe faz uma estrutura genérica em sua inicialização e define as funções que serão utilizadas.
Depois, definimos que para cada camada faremos o uso de nn.Linear, que representa a aplicação da fórmula y = W^T.x + b.
Após isso, definimos como será feito o nosso forward pass: receberemos uma matriz de valores de entrada e faremos um flatten
nela (empilhando suas linhas para formar um vetor), o que gerará nosso vetor de entrada. Em seguida, para cada camada, aplicaremos uma função
de ativação chamada ReLU.


In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        super().__init__()

        # Função de ativação ReLU e camadas lineares (fully connected)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Achatamos os dados de entrada e aplicamos a função ReLU em camadas sucessivas
        flattened_x = self.flatten(x)
        out = self.relu(self.input_layer(flattened_x))
        out = self.relu(self.hidden_layer(out))
        return self.output_layer(out)


## Configuração do Modelo, Função de Custo e Otimizador

Aqui instanciamos nosso modelo NeuralNetwork, definimos a função de custo (CrossEntropyLoss) e o otimizador (Adam).


In [19]:
model = NeuralNetwork(input_size=28*28, hidden_size=64, output_size=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Carregamento dos Dados

Carregamos os datasets MNIST, dividimos em conjuntos de treinamento e teste e transformamos em tensores. 
Em seguida, preparamos os dados com DataLoader para uso na rede.


In [20]:
train_dataset = datasets.MNIST('./', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST('./', train=False, download=True, transform=transforms.ToTensor())

train_dataloader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)


## Configuração de Hiperparâmetros

Aqui definimos o número de épocas (quantas vezes o modelo passará sobre o conjunto de treinamento) e definimos o total de minibatches.


In [21]:
num_epochs = 5
total_batch = len(train_dataloader)


## Treinamento do Modelo

Aqui realizamos o treinamento do modelo, iterando por cada época, calculando gradientes e atualizando pesos.


In [22]:
model.train()
for epoch in range(num_epochs):
    batch_count = 0

    for X, y in train_dataloader:
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        batch_count += 1

        # A cada 300 minibatches, a perda é impressa para monitorar o progresso.
        if batch_count % 300 == 0:
            print(f'epoch {epoch}, step: {batch_count}/{total_batch}: loss: {loss.item()}')


epoch 0, step: 300/938: loss: 0.25446248054504395
epoch 0, step: 600/938: loss: 0.31614920496940613
epoch 0, step: 900/938: loss: 0.16125257313251495
epoch 1, step: 300/938: loss: 0.09956208616495132
epoch 1, step: 600/938: loss: 0.07718589901924133
epoch 1, step: 900/938: loss: 0.12169670313596725
epoch 2, step: 300/938: loss: 0.11994961649179459
epoch 2, step: 600/938: loss: 0.01796025224030018
epoch 2, step: 900/938: loss: 0.12944450974464417
epoch 3, step: 300/938: loss: 0.11952754855155945
epoch 3, step: 600/938: loss: 0.06157275289297104
epoch 3, step: 900/938: loss: 0.02224172092974186
epoch 4, step: 300/938: loss: 0.14250144362449646
epoch 4, step: 600/938: loss: 0.004898659884929657
epoch 4, step: 900/938: loss: 0.12872643768787384


## Teste do Modelo

Aqui fazemos o teste do modelo após o treinamento e calculamos a acurácia.


In [23]:
model.eval()
with torch.no_grad():
    train_acc = []
    test_acc = []

    for X, y in train_dataloader:
        predictions = model(X)
        pred_values, pred_indexes = torch.max(predictions, dim=-1)

        # Cálculo da acurácia comparando as previsões com os rótulos reais.
        acc = sum(pred_indexes == y) / len(y)
        train_acc.append(acc)

    for X, y in test_dataloader:
        predictions = model(X)
        pred_values, pred_indexes = torch.max(predictions, dim=-1)

        # Cálculo da acurácia comparando as previsões com os rótulos reais.
        acc = sum(pred_indexes == y) / len(y)
        test_acc.append(acc)

    print(f'Mean train accuracy: {np.mean(train_acc):.5f}')
    print(f'Mean test accuracy: {np.mean(test_acc):.5f}')


Mean train accuracy: 0.98146
Mean test accuracy: 0.97134
